In [ ]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('.')))

##### **Fetch card images**

In [ ]:
from art_scan import process_query

queries = [
    "Balefire Dragon",
]

[process_query(q) for q in queries];

##### **Pull Private Proxy List from Google Sheets**

In [ ]:
from googleapiclient import discovery
from lib.classes import Proxy
from lib.google_auth import get_creds

args = dict(
    spreadsheetId='13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0',
    range='Inventory!A:L',
)

service = discovery.build('sheets', 'v4', credentials=get_creds())
result = service.spreadsheets().values().get(**args).execute()
proxy_csv_data = list(result.get('values', []))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

##### **Pull Public Proxy List from Google Sheets**

In [ ]:
from lib.classes import Proxy
from lib import common

sheet_id = '13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

proxy_csv_data = common.load_csv(url.format(sheet_id))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

##### **Find missing proxy files**

In [ ]:
from lib import os_helpers

proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

card_types = list(set([p.type for p in proxies_to_print]))
proxies_by_type = {
    type_: [p for p in proxies_to_print if p.type == type_]
    for type_ in card_types
}

missing_files = []
for card_type in card_types:
    folder_name = f"{proxy_folder}/{card_type}"
    proxy_file_names = [p.name for p in proxies_by_type[card_type]]
    missing_files += os_helpers.find_missing_files(folder_name, proxy_file_names)

print("\n".join(missing_files))

##### **Write out files for order**

In [ ]:
# proxies = [p for p in proxies_to_print if not p.is_mdfc]
proxies = [p for p in proxies_to_print if p.is_mdfc]
len(proxies)

In [ ]:
from lib import os_helpers, proxy_helpers
import importlib

importlib.reload(os_helpers)
importlib.reload(proxy_helpers)

folder_size = 612
prefix_name = "proxies_{number:02d}"
landing_folder = "C:/Users/evanh/Temp/landing"
output_folder = "C:/Users/evanh/Temp/mpc-mdfc"
proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
missing, duplicates = proxy_helpers.create_unique_proxies(proxies, proxy_folder, landing_folder)
if len(missing):
    print("Missing:\n\t{}".format('\n\t'.join([p.name for p in missing])))
if len(duplicates):
    print("Duplicates:\n\t{}".format('\n\t'.join([p.name for p in duplicates])))

# Move files into subfolders of fixed size (`folder_size`)
output_folders = os_helpers.split_files(landing_folder, output_folder, folder_size, prefix_name)
output_folders